# (E6) Autoencoders
In this exercise, you will be given an example of [autoencoders](https://en.wikipedia.org/wiki/Autoencoder). 
You should be able to replicate the results given here if you have completed (E2)-(E5) correctly.

It would be best if you have a Python IDE (integrated development environment) such as [PyCharm](https://www.jetbrains.com/pycharm/) and [Anaconda](anaconda.com) is installed because they will make your life easier! If not, you may want to work on the assignment using Google Colab. In any cases, what you need to do is 1) to fill in the blanks in .py files; and 2) to import the files (e.g., layer.py, optim.py, model.py, etc) that you have completed for use. Here are some scenarios how you would go about doing the assignment: 

#### Without Google Colab: Python IDE + Anaconda 
If you have a Python IDE and Anaconda installed, you can do one of the following:
- Edit .py files in the IDE. Then, simply open .ipynb file also in the IDE where you can edit and run codes. 
- Your IDE might not support running .ipynb files. However, since you have installed Anaconda, you can just open this notebook using Jupyter Notebook.

In both of these cases, you can simply import .py files in this .ipynb file:
```python
from model import NeuralNetwork
```
 
#### With Google Colab
- Google Colab has an embedded code editor. So, you could simply upload all .py files to Google Colab and edit the files there. Once you upload the files, double click a file that you want to edit. Please **make sure that you download up-to-date files frequently**, otherwise Google Colab might accidentally restart and all your files might be gone.
- If you feel like the above way is cumbersome, you could instead use any online Python editors for completing .py files (e.g., see [repl.it](https://repl.it/languages/python3)). Also, it's not impossible that you edit the files using any text editors, but they don't show you essential Python grammar information, so you'll be prone to make mistakes in that case. Once you are done editing, you can either upload the files to Colab or follow the instruction below. 
 
- If you have *git clone*d the assignment repository to a directory in your Google Drive (or you have the files stored in the Drive anyway), you can do the following:
```jupyterpython
from google.colab import drive
drive.mount('/content/drive/')          # this will direct you to a link where you can get an authorization key
import sys
sys.path.append('/content/drive/My Drive/your-directory-where-the-python-files-exist')
```
Then, you are good to go. When you change a .py file, make sure it is synced to the drive, then you need to re-run the above lines to get access to the latest version of the file. Note that you should give correct path to *sys.path.append* method.

Now, let's get started!
## Autoencoder
### Input and Target
An autoencoder learns the latent embeddings of inputs in an unsupervised way. This is because we do not need to have specific target values associated with the inputs; however, the input data themselves will act as the targets. 

To see it more concretely, let's look at below code which prepares the data for learning an autoencoder. 

In [1]:
import numpy as np
def generate_data(num=8):
    """ Generate 'num' number of one-hot encoded integers. """ 
    x_train = np.eye(num)[np.arange(num)]                       # This is a simple way to one-hot encode integers
    
    # Repeat x_train multiple times for training
    x_train = np.repeat(x_train, 100, axis=0)
    
    # The target is x_train itself!
    x_target = x_train.copy()
    return x_train, x_target    

Clearly, *x_target* is the same as *x_train*. So, what we want to do is to encode 8-bit inputs using 3 hidden nodes, which in turn will be decoded back to the original 8-bit value by the decoder. Learning an autoencoder, therefore, means that we train both the encoder weight and the decoder weight. In our example, since we have 3 hidden nodes in a single layer, the encoder weight has *[8, 3]* shape, whereas the decoder weight has *[3, 8]* shape. 

### Training an Autoencoder
Now, let us train an autoencoder with the sigmoid activation function and the cross-entropy loss.

In [3]:
from model import NeuralNetwork
from layer import FCLayer
from activation import Activation
from utils import *
from loss import CrossEntropyLoss
from optim import SGD, Adam, RMSProp
# Load data
num = 8
np.random.seed(10)
x_train, x_target = generate_data(num=num)

In [4]:
# Define a model and add fully-connected and activation layers.
nn = NeuralNetwork()
nn.add(FCLayer(x_train.shape[1], 3, initialization='xavier', uniform=False))
nn.add(Activation(sigmoid, sigmoid_prime))
nn.add(FCLayer(3, x_train.shape[1], initialization='xavier', uniform=False))

In [5]:
# Define loss: note that CrossEntropyLoss is using the softmax output internally
loss = CrossEntropyLoss()
nn.set_loss(loss)

In [6]:
# Set up hyperparameters
lr = 0.001
epochs = 2000
freq = epochs // 10
batch_size = 64

In [7]:
# Define optimizer and associate it with the model
optimizer = Adam(nn.parameters(), lr=lr)
nn.set_optimizer(optimizer)

In [8]:
# Training begins
inds = list(range(x_train.shape[0]))
N = x_train.shape[0]

loss_hist = []
for epoch in range(epochs):
    inds = np.random.permutation(inds)
    x_train = x_train[inds]
    x_target = x_target[inds]
    
    loss = 0
    for b in range(0, N, batch_size):
        # get the mini-batch
        x_batch = x_train[b: b+batch_size]
        x_target_batch = x_target[b: b+batch_size]
        
        # feed forward
        pred = nn.predict(x_batch)
        
        # Error
        loss += nn.loss(pred, x_target_batch) / N
        
        # Back propagation of error
        nn.backward(pred, x_target_batch)
        
        # Update parameters
        nn.optimizer.step()
    
    # Record loss per epoch
    loss_hist.append(loss)
    
    if epoch % freq == 0:
        print()
        print("Epoch %d/%d\tloss=%.5f" % (epoch + 1, epochs, loss), end='\t', flush=True)
        
        # Test with the training data
        pred = nn.predict(x_train, mode=False)
        l = nn.loss(pred, x_target)
        print("Test loss: {:.5f}".format(l), end='')

print("\nTraining finished!")
print("Print prediction results:")
x_test = np.eye(num)[np.arange(num)]                        # Test data (one-hot encoded)
np.set_printoptions(2)
for x in x_test:
    print("\tInput: {}\tOutput: {}".format(x, softmax(nn.predict(x[None, :], mode=False))))


Epoch 1/2000	loss=2.13885	Test loss: 1706.35746
Epoch 201/2000	loss=0.72717	Test loss: 579.70946
Epoch 401/2000	loss=0.29857	Test loss: 238.20288
Epoch 601/2000	loss=0.07926	Test loss: 63.18826
Epoch 801/2000	loss=0.02624	Test loss: 20.93281
Epoch 1001/2000	loss=0.00977	Test loss: 7.79089
Epoch 1201/2000	loss=0.00375	Test loss: 2.99201
Epoch 1401/2000	loss=0.00145	Test loss: 1.15597
Epoch 1601/2000	loss=0.00056	Test loss: 0.44764
Epoch 1801/2000	loss=0.00022	Test loss: 0.17384
Training finished!
Print prediction results:
	Input: [1. 0. 0. 0. 0. 0. 0. 0.]	Output: [[1.00e+00 3.17e-15 4.70e-05 1.37e-11 4.63e-06 1.75e-08 2.07e-11 2.43e-05]]
	Input: [0. 1. 0. 0. 0. 0. 0. 0.]	Output: [[3.25e-16 1.00e+00 2.21e-10 1.71e-05 1.10e-09 5.11e-05 1.66e-05 1.16e-10]]
	Input: [0. 0. 1. 0. 0. 0. 0. 0.]	Output: [[3.05e-05 1.81e-10 1.00e+00 3.39e-16 9.14e-10 4.94e-05 2.47e-05 1.65e-10]]
	Input: [0. 0. 0. 1. 0. 0. 0. 0.]	Output: [[1.51e-11 2.54e-05 1.48e-14 1.00e+00 8.08e-06 2.58e-08 2.03e-11 2.42e-05]]


If you look at the output values of the network, clearly we have successfully trained the autoencoder to encode-decode 8-bit integers!

## (E7) Your Turn:  Explain the autoencoder
Given the trained model that can encode the 0-7 integers, explain how the NN model learned to encode/compress the numbers. Rather than just stating your reasoning in words, do explore the model closely to see what it has learned. 